# Dépenses de l'état

Votre première Analyse en composante principale.

**NOTE** : c'est le schéma général que vous devez suivre pour toute ACP.

1. calcul de statistiques descriptives et **interprétations**
2. calcul des données de l'ACP sur des données centrées et réduites
3. étude de l'inertie : répartition de l'inertie pour sur les axes et combien garder d'axes a priori
4. pour les axes d'intérêts :
    1. étude des corrélations des anciens axes avec les nouveaux et **interprétations**
    2. représentations graphiques des projections avec qualités des projections

## Les données

Les données du fichier `dépense_état.csv` décrivent les
 ventilations des dépenses de l'état français de 1872 à 1971. 
Les différents champs sont :

* PVP : pouvoirs publics,
* AGR : agriculture,
* CMI : commerce et industrie,
* TRA : transports,
* LOG : logement et aménagement du territoire,
* EDU : éducation et culture,
* ACO : anciens combattants,
* ACS : actions sociales,
* DEF : défense,
* DET : dette,
* DIV : divers.

In [ ]:
import pandas

dépense = pandas.read_csv("./dépense_état.csv", index_col=0)

In [ ]:
dépense

### Statistiques descriptives

* **Calcul** : 
  * analyse descriptive des données
  * chaque caractéristique étant un pourcentage et chaque ligne sommant à 100%, représentez graphique les lignes en utilisant un [pandas.DataFrame.plot.barh](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.barh.html)
* **Analyses** :
  * Faites des remarques générales sur les statistiques descriptives de ce jeu de donnée.
  * Comment se ventile les caractéristiques par année.
* **Questions** :
  * Que vous apporte les statistiques descriptives de ce jeu de données quand à l'importance de chaque caractéristiques dans le budget de l'état ?
  * Une petite variation est-elle moins importante qu'une grande ?


> Il pourra être utile de consulter les moyens de [visualiser les données en pandas](https://pandas.pydata.org/pandas-docs/stable/visualization.html).

In [ ]:
dépense.describe()

In [ ]:
# dessins matplotlib + stype seaborn

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()


In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

dépense.plot.barh(stacked=True, ax=ax)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)  # en dehors de la figure

plt.show() 

On voit que certaines caractéristiques sont importante, comme `DEF` ou encore `DET` par rapport à `AGR` ou encore `ACS`. On voit émerger des tendances, comme l'augmentation de `ACS` au court du temps et une courbe en "cloche" de `ACO` par exemple.

Une petite variation absolue peut être importante pour une caractéristique qui est elle-même petite. Pensez au budget de la culture par exemple. POur rendre compte des variations de façon relative à la colonne, il est nécessaire de réduire les données.


## ACP

### Calculs

1. centrez et réduisez les données
2. Faites l'ACP en déterminant tous les résultats

In [ ]:
data = dépense

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = pandas.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

In [ ]:
X

In [ ]:
from sklearn.decomposition import PCA 
import numpy as np

pca = PCA() 
pca.fit(X)

U = np.transpose(pca.components_) # vecteurs propres
I = pandas.DataFrame(np.transpose(pca.explained_variance_ratio_), columns=["pourcentage"])  # information véhiculée

C = pandas.DataFrame(X @ U, index=X.index) # nouvelles coordonnées

corrélations = pandas.DataFrame([[C[facteur].corr(X[column]) for facteur in C] for column in X], index=X.columns)
cos2 = (C**2).div((X**2).sum(axis=1), axis='index')

## ACP Interprétations

### Inertie

* **Calculs** :
  * représentez le graphique de l'évolution de l'inertie pour chaque axe. 
  * Représentez également le graphique de l'inertie cumulée.
* **Questions** :
  * Combien d'axes pensez-vous garder ? Pourquoi ?
  * Quels sont les années les moins bien représentées sur ces axes ?

In [ ]:
I

Pour des données centrées réduites le % d'inertie par axe est :

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.plot(ax=ax)

plt.title("inertie expliquée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquée")

plt.show()

In [ ]:

I.cumsum()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.cumsum().plot(ax=ax)

plt.title("inertie expliquée cumulée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquée")

plt.show()

Le point d'inflexion est au alentours du troisième axe (index 2). Le premier axe explique à lui tout seul presque la moitié de l'information (45%). L'inertie totale conservée en gardant 3 axes est de 75% et chaque axe explique plus que 10% de l'information.

On garde donc 3 axes pour l'analyse à priori, mais si on en garde 2 on explique déjà 64% de l'information.

In [ ]:
cos2.iloc[:,:3].sum(axis=1).sort_values()

In [ ]:
from math import sqrt, acos, pi

cos2.iloc[:,:3].sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi).sort_values()

A priori toutes les années sont bien représentées. L'angle max étant de 50°. L'année la moins bien représentée serait l'année 1938 (année particulière s'il en est).

### Interprétation des axes

* **Questions** :
  * proposer une interprétation des 2 premiers axes principaux en utilisant le cercle de corrélation
  * donnez une interprétation des autres axes que vous pensez garder dans l'analyse

#### Axes 1 et 2

In [ ]:
axe_x = 0
axe_y = 1

couleurs = [sns.color_palette()[3]] * len(corrélations)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = corrélations.plot.scatter(x=axe_x, y=axe_y, ax=ax, 
                                 color=couleurs)


for index, row in corrélations.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))

draw.add_patch(plt.Circle((0, 0), radius=1, color=(0, 0, 0, .3)))

plt.xlim(-1.2, 1.2)
plt.ylim(-1.2, 1.2)
plt.axvline(0)
plt.axhline(0)

plt.show()

On ne regarde que les caractéristiques qui **contribuent le plus** (en positif ou en négatif) pour chaque axe. On cherche soit les tendances soit les oppositions entre les caractéristiques :

* axe 0 (horizontal) : on oppose `DET` à `EDU`, `ACS`, `AGR`, `CMI` et `LOG` 
* axe 1 (vertical) : on oppose `ACO` à `TRA` et `PVP`

Le 1er axe est une opposition entre un remboursement de la dette (à gauche) et des dépenses liées à l'action sociale, l'éducation, logement, commerce et industrie. Bref entre le passé (passif ?) et le présent/futur de la nation. Le second axe oppose quant à lui les anciens combatants au travaux publics (semble bizarre à première vue).

#### Axe 3

On représente les corrélations pour l'axe d'indice 2

In [ ]:
corrélations[2].sort_values()

Les corrélations ne sont pas aussi tranchées pour les 2 premiers axe. C'est normal, il n'explique que peu d'information (10%) par rapport à eux. 

Si l'on garde les caractéristiques de corrélations maximum en valeur absolue, seule `DIV` sort du lot avec un score de .54

On peut interpréter cet axe comme un axe où divers est plus important que d'habitude. Reste à voir les années qui se projettent bien sur cet axe.

### Projections

* **Calculs** :
  * représentez graphiquement les projections des années sur les 2 premiers axes factoriels
  * quels sont les années qui ne se représentent pas bien dessus ?
  * quels snt les années qui se représentent bien sur les autres axes que vous avez descidé de garder ?
* **Interprétations** :
  * Interprétez la répartition des années sur ce graphique. Peut-on grouper les années par paquet homogène ?
  * Interprétez la répartition des années sur les autres axes.

In [ ]:
C.iloc[:,:3]

In [ ]:
axe_x = 0
axe_y = 1

couleurs = [sns.color_palette()[0]] * len(C)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, color=couleurs , ax=ax)

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))
    
plt.axvline(0)
plt.axhline(0)

plt.show() 

In [ ]:
cos2.iloc[:,:2].sum(axis=1).sort_values().head(5)

In [ ]:
from math import sqrt, acos, pi

cos2.iloc[:,:2].sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi).sort_values().tail(5)

Les années se représentent assez bien. L'angle maximum étant de 53°. Au pire, ce sont les années 1938 et 1953 que l'on pourrait retoquer car l'angle est supérieur à 50°.

1. On remarque que les années se projettent de façon chronologique selon l'axe 0 : la tendance du budget de l'état français au cours du temps est de suivre les tendances décrites par le 1er axe factoriel
2. on peut regrouper les années en 4 grands groupes :
   * à gauche : les années avant la 1ère guerre mondiale
   * en haut au milieu : l'entre 2 guerre
   * à droite : post 2ne guerre mondiale
   * en bas : immédiate après-guerre

### axe 2 (troisième axe)

In [ ]:
I.loc[2]

Les années qui sont le plus représentées par cet axe sont :

In [ ]:
cos2.iloc[:, 2].sort_values().tail(10)

In [ ]:
from math import sqrt, acos, pi

cos2.iloc[:, 2].apply(lambda x: acos(sqrt(x)) * 180 / pi).sort_values().head(10)

Projections axe 0 et 2

In [ ]:
axe_x = 0
axe_y = 2

couleurs = [sns.color_palette()[0]] * len(C)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, color=couleurs , ax=ax)

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))
    
plt.axvline(0)
plt.axhline(0)

plt.show() 